In [10]:
# Importing all dependency
import numpy as np
import pickle
import heapq
from collections import deque
import time

In [11]:
# Load Data Files
adj_matrix = np.load('IIIT_Delhi.npy')
with open('IIIT_Delhi.pkl', 'rb') as f:
    node_attributes = pickle.load(f)
    
print(adj_matrix)
print()
print(node_attributes)

[[ 0.     0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]
 ...
 [ 0.     0.     0.    ...  0.    37.654  0.   ]
 [ 0.     0.     0.    ... 37.654  0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]]

{0: {'x': 77.2770643, 'y': 28.5477148}, 1: {'x': 77.2698125, 'y': 28.5503445}, 2: {'x': 77.2694371, 'y': 28.5488449}, 3: {'x': 77.269063, 'y': 28.5477874}, 4: {'x': 77.2681825, 'y': 28.5494218}, 5: {'x': 77.2713275, 'y': 28.5489937}, 6: {'x': 77.269159, 'y': 28.5495859}, 7: {'x': 77.2690067, 'y': 28.5502523}, 8: {'x': 77.270938, 'y': 28.5496351}, 9: {'x': 77.2702386, 'y': 28.5492781}, 10: {'x': 77.2732571, 'y': 28.5500462}, 11: {'x': 77.2687051, 'y': 28.5484867}, 12: {'x': 77.2737348, 'y': 28.5493724}, 13: {'x': 77.2723301, 'y': 28.5470585}, 14: {'x': 77.2691629, 'y': 28.543878}, 15: {'x': 77.269829, 'y': 28.5463013}, 16: {'x': 77.2685016, 'y': 28.5503304}, 17: {'x': 77.2762048, 'y': 28.543

In [12]:
def get_ids_path(adj_matrix, start_node, goal_node):
  start_time = time.time()
  timeout = 3
  def ids(rem_node, live_node, goal_node, visited_node):
    if time.time() - start_time > timeout:
      return None
    
    if rem_node == 0:
      return None
    
    if live_node == goal_node:
      return [live_node]
    
    visited_node.add(live_node)
    
    for nb in range(len(adj_matrix)):
      if adj_matrix[live_node][nb] > 0 and nb not in visited_node:
        if trav_path := ids(rem_node - 1, nb, goal_node, visited_node.copy()) :
          return [live_node] + trav_path
      
    return None
  
  for total_traversal in range(len(adj_matrix)):
    visited_node = set()
    if trav_path:=ids(total_traversal, start_node, goal_node, visited_node) :
      return trav_path
    
  return None


In [13]:
# Implementing Bi-directional search

def get_bidirectional_search_path(adj_matrix, start_node, goal_node):
  if start_node == goal_node:
      return [start_node]
  
  # Maintaining queue of traversing node from start and goal nodes
  queue_for_start_node = deque([(start_node, [start_node])])
  queue_for_goal_node = deque([(goal_node, [goal_node])])
  
  visit_node_from_start = {start_node: [start_node]}
  visit_node_from_goal = {goal_node: [goal_node]}
  
  while queue_for_start_node and queue_for_goal_node:
      # Traversing from starting node
      next_node, trav_path = queue_for_start_node.popleft()
      for nearest, path_cost in enumerate(adj_matrix[next_node]):
          if path_cost > 0 and nearest not in visit_node_from_start:
              visit_node_from_start[nearest] = trav_path + [nearest]
              queue_for_start_node.append((nearest, visit_node_from_start[nearest]))
              if nearest in visit_node_from_goal:
                  return visit_node_from_start[nearest] + visit_node_from_goal[nearest][::-1][1:]
      
      # Traversing from goal node
      next_node, trav_path = queue_for_goal_node.popleft()
      for nearest, path_cost in enumerate(adj_matrix[next_node]):
          if path_cost > 0 and nearest not in visit_node_from_goal:
              visit_node_from_goal[nearest] = trav_path + [nearest]
              queue_for_goal_node.append((nearest, visit_node_from_goal[nearest]))
              if nearest in visit_node_from_start:
                  return visit_node_from_goal[nearest] + visit_node_from_start[nearest][::-1][1:]
  
  return None


In [14]:
# A* Implementation

def get_astar_search_path(adj_matrix, node_attributes, start_node, goal_node):
    def euclidean_distance(point1, point2):
        if point1['x'] > point2['x']: x = point1['x'] - point2['x']
        else: x = point2['x'] - point1['x']
        if point1['y'] > point2['y']: y = point1['y'] - point2['y']
        else: y = point2['y'] - point1['y']
        return round((x**2 + y**2) ** 0.5, 5)
    
    def heuristic_cost(node):
        to_node = euclidean_distance(node_attributes[start_node], node_attributes[node])
        to_goal = euclidean_distance(node_attributes[node], node_attributes[goal_node])
        return  round(to_goal + to_node, 5)
    
    h_cost = {}
    for i in range(len(adj_matrix)):
        h_cost[i] = heuristic_cost(i)
    
    live_nodes = [(h_cost[start_node], 0, start_node, [start_node])]
    visited_node = set()
    g_costs = {start_node: 0}
    best_paths = {start_node: [start_node]}

    while live_nodes:
        f_cost, g_cost, current_node, path = heapq.heappop(live_nodes)
        if current_node == goal_node:
            return path
        
        if current_node in visited_node:
            continue
        visited_node.add(current_node)

        for nearest, n_cost in enumerate(adj_matrix[current_node]):
            if n_cost > 0 and nearest not in visited_node:
                u_cost = g_cost + n_cost
                if nearest not in g_costs or u_cost < g_costs[nearest]:
                    g_costs[nearest] = u_cost
                    f_cost = u_cost + h_cost[nearest]
                    best_paths[nearest] = path + [nearest]
                    heapq.heappush(live_nodes, (f_cost, u_cost, nearest, best_paths[nearest]))

    return None

In [15]:
# Implementation of Bi-directional Heuristic search


def get_bidirectional_heuristic_search_path(adj_matrix, node_attributes, start_node, goal_node):
  def euclidean_distance(point1, point2):
        if point1['x'] > point2['x']: x = point1['x'] - point2['x']
        else: x = point2['x'] - point1['x']
        if point1['y'] > point2['y']: y = point1['y'] - point2['y']
        else: y = point2['y'] - point1['y']
        return round((x**2 + y**2) ** 0.5, 5)
    
  def heuristic_cost(node):
      to_node = euclidean_distance(node_attributes[start_node], node_attributes[node])
      to_goal = euclidean_distance(node_attributes[node], node_attributes[goal_node])
      return  round(to_goal + to_node, 5)
    
  h_cost = {}
  for i in range(len(adj_matrix)):
      h_cost[i] = heuristic_cost(i)
    
  # Maintaining queue of traversing node from start and goal nodes
  queue_for_start_node = [(0 + h_cost[start_node], 0, start_node, [start_node])]
  queue_for_goal_node = [(0 + h_cost[goal_node], 0, goal_node, [goal_node])]

  visit_node_from_start = {start_node: (0, [start_node])}
  visit_node_from_goal = {goal_node: (0, [goal_node])}

  while queue_for_start_node and queue_for_goal_node:
      # Traversing from starting node
      f_cost_start, g_cost_start, current_node_start, path_from_start = heapq.heappop(queue_for_start_node)
      for nearest, path_cost in enumerate(adj_matrix[current_node_start]):
          if path_cost > 0:
              g_new = g_cost_start + path_cost
              if nearest not in visit_node_from_start or g_new < visit_node_from_start[nearest][0]:
                  visit_node_from_start[nearest] = (g_new, path_from_start + [nearest])
                  f_new = g_new + h_cost[nearest]
                  heapq.heappush(queue_for_start_node, (f_new, g_new, nearest, path_from_start + [nearest]))

                  if nearest in visit_node_from_goal:
                      return visit_node_from_start[nearest][1] + visit_node_from_goal[nearest][1][::-1][1:]

      # Traversing from goal node
      f_cost_goal, g_cost_goal, current_node_goal, path_from_goal = heapq.heappop(queue_for_goal_node)
      for nearest, path_cost in enumerate(adj_matrix[current_node_goal]):
          if path_cost > 0:
              g_new = g_cost_goal + path_cost
              if nearest not in visit_node_from_goal or g_new < visit_node_from_goal[nearest][0]:
                  visit_node_from_goal[nearest] = (g_new, path_from_goal + [nearest])
                  f_new = g_new + h_cost[nearest]
                  heapq.heappush(queue_for_goal_node, (f_new, g_new, nearest, path_from_goal + [nearest]))

                  if nearest in visit_node_from_start:
                      return visit_node_from_goal[nearest][1] + visit_node_from_start[nearest][1][::-1][1:]

  return None


In [16]:
def bonus_problem(adj_matrix):
    def find_removal(u, parent_node):
        nonlocal vistors_t
        is_node_visited[u] = True
        when_tra[u] = vistors_t
        min_trav[u] = vistors_t
        vistors_t += 1

        for v in range(len(adj_matrix)):
            if adj_matrix[u][v] != 0:
                if not is_node_visited[v]:
                    parent_node[v] = u
                    find_removal(v, parent_node)
                    min_trav[u] = min(min_trav[u], min_trav[v])

                    if min_trav[v] > when_tra[u]:
                        path_removalble.append((u, v))
                elif v != parent_node[u]:
                    min_trav[u] = min(min_trav[u], when_tra[v])

    no_nodes = len(adj_matrix)
    is_node_visited = [False] * no_nodes
    when_tra = [-1] * no_nodes
    min_trav = [-1] * no_nodes
    parent_node = [-1] * no_nodes
    path_removalble = []
    vistors_t = 0

    for i in range(no_nodes):
        if not is_node_visited[i]:
            find_removal(i, parent_node)

    return path_removalble

In [18]:
start_node = int(input("Enter the start node: "))
end_node = int(input("Enter the end node: "))

print(f'Iterative Deepening Search Path: {get_ids_path(adj_matrix,start_node,end_node)}')
print(f'Bidirectional Search Path: {get_bidirectional_search_path(adj_matrix,start_node,end_node)}')
print(f'A* Path: {get_astar_search_path(adj_matrix,node_attributes,start_node,end_node)}')
print(f'Bidirectional Heuristic Search Path: {get_bidirectional_heuristic_search_path(adj_matrix,node_attributes,start_node,end_node)}')
print(f'Bonus Problem: {bonus_problem(adj_matrix)}')

Iterative Deepening Search Path: [5, 97, 98, 12]
Bidirectional Search Path: [5, 97, 28, 10, 12]
A* Path: [5, 97, 28, 10, 12]
Bidirectional Heuristic Search Path: [5, 97, 28, 10, 12]
Bonus Problem: [(42, 29), (42, 30), (113, 42), (113, 43), (15, 46), (35, 15), (114, 84), (36, 114), (38, 36), (87, 88), (69, 124), (41, 70), (45, 17), (89, 90), (51, 50), (39, 40), (73, 72), (19, 100), (106, 107), (108, 109), (108, 112), (111, 108), (111, 110), (106, 111), (75, 106), (55, 56), (12, 57), (53, 54), (95, 96), (53, 95), (14, 53), (14, 99), (47, 48)]


In [9]:
import psutil
import os
import time

def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss  # Return the memory usage in bytes

def run_and_compare_with_memory(graph, test_cases):
    for src, dest in test_cases:
        path_ids = get_ids_path(graph, src, dest)

test_cases = [(i, j) for i in range(125) for j in range(125)]
# Run comparison

start_mem = memory_usage()
start_time = time.time()
run_and_compare_with_memory(adj_matrix, test_cases)
bfs_time = (time.time() - start_time) * 1000
bfs_mem = memory_usage() - start_mem

print(bfs_mem, bfs_time)

KeyboardInterrupt: 

In [ ]:
import psutil
import os
import time

def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss  # Return the memory usage in bytes

def run_and_compare_with_memory(graph, node_attributes, test_cases):
    for src, dest in test_cases:
        # Measure time and memory for IDS
        start_mem = memory_usage()
        start_time = time.time()
        path_ids = get_astar_search_path(graph, node_attributes, src, dest)
        ids_time = (time.time() - start_time) * 1000
        ids_mem = memory_usage() - start_mem

        # Measure time and memory for Bidirectional BFS
        start_mem = memory_usage()
        start_time = time.time()
        path_bfs = get_bidirectional_heuristic_search_path(graph, node_attributes, src, dest)
        bfs_time = (time.time() - start_time) * 1000
        bfs_mem = memory_usage() - start_mem

        print(f"Source: {src}, Destination: {dest}")
        print(f"Astar Search Path: {path_ids}, Time: {ids_time} ms, Memory: {ids_mem / 1024} KB")
        print(f"Bidirectional Heuristic Search Path: {path_bfs}, Time: {bfs_time} ms, Memory: {bfs_mem / 1024} KB")
        print('-' * 50)

test_cases = [(1, 2), (5, 12), (12, 49), (4, 12), (7, 12)]

# Run comparison
run_and_compare_with_memory(adj_matrix, node_attributes, test_cases)


In [ ]:
import matplotlib.pyplot as plt

# Data for scatter plot
# Time in milliseconds, Memory in bytes, Path Length (Number of nodes in the path)
data = {
    'Iterative Deepening': {
        'times': [1.9958, 5.0025, 15001.4126, 26.0026, 38.0013],
        'memory': [0, 0, 32768, 0, 0],  # 32 KB converted to 32768 bytes
        'path_length': [4, 4, 0, 9, 9]  # 0 indicates no path found
    },
    'Bidirectional Search': {
        'times': [0.0, 0.98395, 0.0, 1.0011, 1.0009],
        'memory': [0, 0, 0, 0, 0],
        'path_length': [4, 5, 0, 9, 9]
    },
    'Astar Search': {
        'times': [1.9989, 2.9995, 7.9992, 1.9984, 3.9992],
        'memory': [0, 0, 61440, 0, 0],  # 60 KB converted to 61440 bytes
        'path_length': [3, 5, 0, 10, 9]
    },
    'Bidirectional Heuristic': {
        'times': [1.0004, 1.0002, 1.0011, 2.0003, 4.0002],
        'memory': [0, 0, 0, 0, 0],
        'path_length': [4, 5, 0, 10, 9]
    }
}

# Plot 1: Time vs Memory (now in bytes and milliseconds)
plt.figure(figsize=(10, 6))
for algo, values in data.items():
    plt.scatter(values['memory'], values['times'], label=algo)

plt.title('Time (ms) vs Memory (bytes) (Efficiency Analysis)')
plt.xlabel('Memory (bytes)')
plt.ylabel('Time (milliseconds)')
plt.legend()
plt.grid(True)
plt.show()

# Plot 2: Path Length vs Time (Optimality)
plt.figure(figsize=(10, 6))
for algo, values in data.items():
    plt.scatter(values['path_length'], values['times'], label=algo)

plt.title('Path Length (Number of Nodes) vs Time (milliseconds) (Optimality Analysis)')
plt.xlabel('Path Length (Number of Nodes)')
plt.ylabel('Time (milliseconds)')
plt.legend()
plt.grid(True)
plt.show()
